<a href="https://colab.research.google.com/github/kanri3/deep_learning_day1_day2/blob/main/2_3_2_dropout_hands_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

## Googleドライブのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## sys.pathの設定

以下では，Googleドライブのマイドライブ直下にDNN_codeフォルダを置くことを仮定しています．必要に応じて，パスを変更してください．

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/DNN_code')
# sys.path.append('/content/drive/My Drive/DNN_code/lesson_2')

In [ ]:
import numpy as np
from collections import OrderedDict
from common import layers, optimizer
from data.mnist import load_mnist
import matplotlib.pyplot as plt
# from common.multi_layer_net import MultiLayerNet
# from multi_layer_net import MultiLayerNet
# from common import optimizer
import copy

(x_train, tch_train), (x_test, tch_test) = load_mnist(normalize = True)
print("x_train.shape :", x_train.shape)
print("データ読み込み完了")

# 過学習を再現するために、学習データを削減
# x_train = x_train[:200]
# tch_train = tch_train[:200]
x_train = x_train[:300]
tch_train = tch_train[:300]
print("x_train :", x_train)

# network_init = MultiLayerNet(input_size=784, hidden_size_list = [100, 100, 100, 100, 100, 100], output_size = 10)

optimizer_SDG_init = optimizer.SGD(learning_rate = 0.01)

iters_num = 5
# iters_num = 1000
train_size = x_train.shape[0]
# batch_size = 10
batch_size = 100
# plot_interval=50
plot_interval=100

# グラフ
# def draw_graph(title, legend_position):
#     lists = range(0, iters_num + 1, plot_interval)
#     # lists = range(0, iters_num, plot_interval)
#     # print("lists, accuracies_train :", lists, accuracies_train)
#     plt.plot(lists, accuracies_train, label="training set")
#     plt.plot(lists, accuracies_test,  label="test set")
#     plt.legend(loc = legend_position)
#     # plt.legend(loc="lower right")
#     plt.title(title)
#     plt.xlabel("count")
#     plt.ylabel("accuracy")
#     plt.ylim(0, 1.0)
#     # グラフの表示
#     plt.show()

x_train.shape : (60000, 784)
データ読み込み完了
x_train : [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Multi Layer Net

In [ ]:
import sys, os
sys.path.append(os.pardir) # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from collections import OrderedDict
from common import layers as lyrs

class MultiLayerNet:
    '''
    input_size: 入力層のノード数
    hidden_size_list: 隠れ層のノード数のリスト
    output_size: 出力層のノード数
    activation: 活性化関数
    weight_init_std: 重みの初期化方法
    weight_decay_lambda: L2正則化の強さ
    use_dropout: ドロップアウトの有無
    dropout_ratio: ドロップアウト率
    use_batchnorm: バッチ正規化の有無
    '''
    def __init__(self, input_size, hidden_size_list, output_size, activation='relu', weight_init_std='relu', weight_decay_lambda=0,
                 use_dropout = False, dropout_ratio = 0.5, use_batchnorm=False):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size_list = hidden_size_list
        self.hidden_layer_num = len(hidden_size_list)
        self.use_dropout = use_dropout
        self.weight_decay_lambda = weight_decay_lambda
        self.use_batchnorm = use_batchnorm
        self.params = {}

        # 重みの初期化
        self.__init_weight(weight_init_std)

        # レイヤの生成
        activation_layer = {'sigmoid': lyrs.Sigmoid, 'relu': lyrs.Relu}
        self.layers = OrderedDict()
        print("self.hidden_layer_num :", self.hidden_layer_num)
        for idx in range(1, self.hidden_layer_num + 1):
            self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)], self.params['b' + str(idx)])
            # self.layers['Affine' + str(idx)] = lyrs.Affine(self.params['W' + str(idx)], self.params['b' + str(idx)])
            if self.use_batchnorm:
                self.params['gamma' + str(idx)] = np.ones(hidden_size_list[idx - 1])
                self.params['beta' + str(idx)] = np.zeros(hidden_size_list[idx - 1])
                self.layers['BatchNorm' + str(idx)] = lyrs.BatchNormalization(self.params['gamma' + str(idx)], self.params['beta' + str(idx)])
                
            self.layers['Activation_function' + str(idx)] = activation_layer[activation]()
            
            if self.use_dropout:
                self.layers['Dropout' + str(idx)] = Dropout(dropout_ratio)
                # self.layers['Dropout' + str(idx)] = lyrs.Dropout(dropout_ratio)

        idx = self.hidden_layer_num + 1
        self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)], self.params['b' + str(idx)])
        # self.layers['Affine' + str(idx)] = lyrs.Affine(self.params['W' + str(idx)], self.params['b' + str(idx)])

        self.last_layer = lyrs.SoftmaxWithLoss()
        print("self.layers.items() :", self.layers.items())
        print("len self.layers :", len(self.layers))

    def __init_weight(self, weight_init_std):
        all_size_list = [self.input_size] + self.hidden_size_list + [self.output_size]
        print("all_size_list", all_size_list)
        for idx in range(1, len(all_size_list)):
            scale = weight_init_std
            if str(weight_init_std).lower() in ('relu', 'he'):
                scale = np.sqrt(2.0 / all_size_list[idx - 1])  # ReLUを使う場合に推奨される初期値
            elif str(weight_init_std).lower() in ('sigmoid', 'xavier'):
                scale = np.sqrt(1.0 / all_size_list[idx - 1])  # sigmoidを使う場合に推奨される初期値
            self.params['W' + str(idx)] = scale * np.random.randn(all_size_list[idx-1], all_size_list[idx])
            self.params['b' + str(idx)] = np.zeros(all_size_list[idx])
            print("W", idx, " : ", self.params['W' + str(idx)].shape)
            print("b", idx, " : ", self.params['b' + str(idx)].shape)

    def predict(self, x_tr, train_flg = False):
      print("x_tr.shape まるれいpredi :", x_tr.shape)
      for key, layer in self.layers.items():
          if "Dropout" in key or "BatchNorm" in key:
              print("x_tr.shape まるれいpredi ifない :", x_tr.shape)
              x_tr = layer.forward(x_tr, train_flg)
          else:
              x_tr = layer.forward(x_tr)
      return x_tr

    def loss(self, x_tr, tch_tr, train_flg = False):
      print("x_tr.shape まるれいloss :", x_tr.shape)
      y = self.predict(x_tr, train_flg)
      weight_decay = 0
      for idx in range(1, self.hidden_layer_num + 2):
          W = self.params['W' + str(idx)]
          weight_decay += 0.5 * self.weight_decay_lambda * np.sum(W ** 2)
      print("loss メソッドのreturn前")
      return self.last_layer.forward(y, tch_tr) + weight_decay

    def accuracy(self, X, D):
        Y = self.predict(X, train_flg=False)
        Y = np.argmax(Y, axis=1)
        if D.ndim != 1 : D = np.argmax(D, axis=1)
        accuracy = np.sum(Y == D) / float(X.shape[0])
        return accuracy

    def gradient(self, x_tr, tch_tr):
        # forward
        print("x_tr.shape まるれいgrad :", x_tr.shape)
        self.loss(x_tr, tch_tr, train_flg = True)
        # backward
        dout = 1
        dout = self.last_layer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        # 設定
        grads = {}
        for idx in range(1, self.hidden_layer_num + 2):
            grads['W' + str(idx)] = self.layers['Affine' + str(idx)].dW + self.weight_decay_lambda * self.params['W' + str(idx)]
            grads['b' + str(idx)] = self.layers['Affine' + str(idx)].db

            if self.use_batchnorm and idx != self.hidden_layer_num + 1:
                grads['gamma' + str(idx)] = self.layers['BatchNorm' + str(idx)].dgamma
                grads['beta' + str(idx)] = self.layers['BatchNorm' + str(idx)].dbeta
        return grads

## Dropout

In [ ]:
class Dropout:
    def __init__(self, dropout_ratio = 0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x_tr, train_flg = True):
        print("x_tr.shape Dropout.forw():", x_tr.shape)
        print("train_flg :", train_flg)
        if train_flg:
            # 引数の前のアスタリスクは可変長引数
            # https://dev.classmethod.jp/articles/what-does-asterisk-mean-at-args/
            self.mask = np.random.rand(*x_tr.shape) > self.dropout_ratio
            # print("np.random.rand(*x_tr.shape) :", np.random.rand(*x_tr.shape))
            print("self.mask.shape :", self.mask.shape)
            returned_value = x_tr * self.mask
            print("x_tr.shape :", x_tr.shape)
            print("(x_tr * self.mask).shape :", returned_value.shape)
        else:
            returned_value = x_tr * (1.0 - self.dropout_ratio)
            # print("x_tr * (1.0 - self.dropout_ratio) :", x_tr * (1.0 - self.dropout_ratio))
        return returned_value

    def backward(self, dout):
        return dout * self.mask

## Affine

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        print("aFFFFFFine")
        self.x = None
        self.original_x_shape = None
        # 重み・バイアスパラメータの微分
        self.dW = None
        self.db = None

    def forward(self, x):
        # テンソル対応        
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        out = np.dot(self.x, self.W) + self.b
        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        dx = dx.reshape(*self.original_x_shape)  # 入力データの形状に戻す（テンソル対応）
        return dx

## 実行部

In [ ]:
from common import optimizer

# ドロップアウト設定 ======================================
use_dropout = True
# dropout_ratio = 0.3
dropout_ratio = 0.08
# ====================================================

network = MultiLayerNet(input_size=784, 
                        hidden_size_list=[5, 5], 
                        # hidden_size_list=[100, 100, 100, 100, 100, 100], 
                        output_size=10,
                        # weight_decay_lambda=weight_decay_lambda, 
                        use_dropout = use_dropout, dropout_ratio = dropout_ratio)
optimizer = optimizer.SGD(learning_rate=0.01)
# optimizer = optimizer.Momentum(learning_rate=0.01, momentum=0.9)
# optimizer = optimizer.AdaGrad(learning_rate=0.01)
# optimizer = optimizer.Adam()

train_loss_list = []
accuracies_train = []
accuracies_test = []
# plot_interval=10

for i in range(iters_num):
    print("iter loop_val i=", i)
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    tch_batch = tch_train[batch_mask]
    print("batch_mask :", batch_mask)
    print("x_train.shape :", x_train.shape)
    print("x_batch.shape :", x_batch.shape)

    print("x_batch.shape network.graaaa():", x_batch.shape)
    grad = network.gradient(x_batch, tch_batch)
    optimizer.update(network.params, grad)

    print("iter network.loss(")
    print("x_batch.shape network.loss():", x_batch.shape)
    loss = network.loss(x_batch, tch_batch)
    train_loss_list.append(loss)    
    
    # if (i + 1) % plot_interval == 0 or i == 0:
    #     accr_train = network.accuracy(x_train, tch_train)
    #     accr_test = network.accuracy(x_test, tch_test)
    #     accuracies_train.append(accr_train)
    #     accuracies_test.append(accr_test)

    #     print('Generation: ' + str(i+1) + '. 正答率(トレーニング) = ' + str(format(accr_train,'.5f')))
    #     print('                : ' + str(i+1) + '. 正答率(テスト) = ' + str(format(accr_test,'.5f')))        

# グラフの表示
# draw_graph("drop out!!", "lower right")

all_size_list [784, 5, 5, 10]
W 1  :  (784, 5)
b 1  :  (5,)
W 2  :  (5, 5)
b 2  :  (5,)
W 3  :  (5, 10)
b 3  :  (10,)
self.hidden_layer_num : 2
aFFFFFFine
aFFFFFFine
aFFFFFFine
self.layers.items() : odict_items([('Affine1', <__main__.Affine object at 0x7efebd563450>), ('Activation_function1', <common.layers.Relu object at 0x7efebd563f50>), ('Dropout1', <__main__.Dropout object at 0x7efebd563b50>), ('Affine2', <__main__.Affine object at 0x7efebd563b10>), ('Activation_function2', <common.layers.Relu object at 0x7efebd563b90>), ('Dropout2', <__main__.Dropout object at 0x7efebd563d90>), ('Affine3', <__main__.Affine object at 0x7efebd563e90>)])
len self.layers : 7
iter loop_val i= 0
batch_mask : [215  50 109 275  44 298  12 121  25  35 209  24  67  37 149 132  79 164
 229 281 291  67 265 138  56 288 174  94  76 164 243  10 268  41 235 235
 183 150  23 127  33 225 267  64  81 192 154 195  84 213 271 110 272  92
 255 225 153 125 157 233 110 104 158 142 257 185 106  43 183 103 210 257
 240 275

In [ ]:
mat_01 = np.array([[1, 2], [3, 4]])
mat_bool = np.array([[True, False], [True, False]])
mat_01 * mat_bool

array([[1, 0],
       [3, 0]])